In [17]:
import pandas as pd

# Load the combined climate and device data
climate_data = pd.read_csv('training/clean/combined_climate_data.csv')
device_data = pd.read_csv('training/clean/combined_device_data.csv')

# Convert the 'Time' and 'time' columns to datetime format
climate_data['Time'] = pd.to_datetime(climate_data['Time'])
device_data['time'] = pd.to_datetime(device_data['time'])

# Merge the datasets on the timestamp
merged_df = pd.merge(device_data, climate_data, left_on='time', right_on='Time', how='inner')

# Drop the redundant 'Time' column
merged_df.drop(columns=['Time'], inplace=True)

# Add time-based features
merged_df['hour'] = merged_df['time'].dt.hour
merged_df['day'] = merged_df['time'].dt.day
merged_df['month'] = merged_df['time'].dt.month

# Check for NaN values
print("NaN values in merged_df:\n", merged_df.isna().sum())

# Drop rows with NaN values or handle them appropriately
merged_df = merged_df.dropna()

# Print the merged dataframe
print(merged_df.head())
print("Column names:", merged_df.columns)

# Select features and target variable
features = ['Temperature', 'Humidity', 'hour', 'day', 'month', 'temperature']  # Include device temperature in features
target = 'temperature'


NaN values in merged_df:
 time           0
temperature    1
Temperature    0
Humidity       0
hour           0
day            0
month          0
dtype: int64
                       time  temperature  Temperature  Humidity  hour  day  \
0 2023-03-15 15:00:00+08:00    20.238870           31      0.52    15   15   
1 2023-03-15 16:00:00+08:00    20.248299           31      0.52    16   15   
2 2023-03-15 17:00:00+08:00    20.249218           30      0.55    17   15   
3 2023-03-15 18:00:00+08:00    20.248273           30      0.59    18   15   
4 2023-03-15 19:00:00+08:00    20.172554           29      0.58    19   15   

   month  
0      3  
1      3  
2      3  
3      3  
4      3  
Column names: Index(['time', 'temperature', 'Temperature', 'Humidity', 'hour', 'day',
       'month'],
      dtype='object')


In [18]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Scale the features and target separately
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

scaled_features = feature_scaler.fit_transform(merged_df[features])
scaled_target = target_scaler.fit_transform(merged_df[[target]])

# Check for NaN values in scaled features and target
print("NaN values in scaled_features:", np.isnan(scaled_features).sum())
print("NaN values in scaled_target:", np.isnan(scaled_target).sum())

# Prepare sequences for LSTM
def create_sequences(features, target, seq_length):
    xs = []
    ys = []
    for i in range(len(features) - seq_length):
        x = features[i:i+seq_length]
        y = target[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, y = create_sequences(scaled_features, scaled_target, seq_length)

# Check for NaN values in sequences
print("NaN values in X:", np.isnan(X).sum())
print("NaN values in y:", np.isnan(y).sum())

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape input to be [samples, time steps, features]
X_train = X_train.reshape((X_train.shape[0], seq_length, len(features)))
X_test = X_test.reshape((X_test.shape[0], seq_length, len(features)))

# Check for NaN values in training and testing sets
print("NaN values in X_train:", np.isnan(X_train).sum())
print("NaN values in X_test:", np.isnan(X_test).sum())
print("NaN values in y_train:", np.isnan(y_train).sum())
print("NaN values in y_test:", np.isnan(y_test).sum())


NaN values in scaled_features: 0
NaN values in scaled_target: 0
NaN values in X: 0
NaN values in y: 0
NaN values in X_train: 0
NaN values in X_test: 0
NaN values in y_train: 0
NaN values in y_test: 0


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define a more complex LSTM model
model = Sequential([
    LSTM(128, activation='relu', return_sequences=True, input_shape=(seq_length, len(features))),  # Increased number of units
    Dropout(0.3),  # Increased dropout rate
    LSTM(64, activation='relu'),  # Second LSTM layer with more units
    Dropout(0.3),  # Increased dropout rate
    Dense(32, activation='relu'),  # Added a Dense layer
    Dense(1)  # Output layer
])

# Compile the model with a different optimizer and learning rate
optimizer = Adam(learning_rate=0.001)  # Specifying learning rate
model.compile(optimizer=optimizer, loss='mse')

# Train the model with modified batch size and number of epochs
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
15/15 [==============================] - 2s 25ms/step - loss: 0.1503 - val_loss: 0.0735
Epoch 2/100
15/15 [==============================] - 0s 11ms/step - loss: 0.0794 - val_loss: 0.0593
Epoch 3/100
15/15 [==============================] - 0s 14ms/step - loss: 0.0578 - val_loss: 0.0504
Epoch 4/100
15/15 [==============================] - 0s 10ms/step - loss: 0.0476 - val_loss: 0.0401
Epoch 5/100
15/15 [==============================] - 0s 13ms/step - loss: 0.0458 - val_loss: 0.0366
Epoch 6/100
15/15 [==============================] - 0s 10ms/step - loss: 0.0396 - val_loss: 0.0396
Epoch 7/100
15/15 [==============================] - 0s 11ms/step - loss: 0.0364 - val_loss: 0.0347
Epoch 8/100
15/15 [==============================] - 0s 10ms/step - loss: 0.0322 - val_loss: 0.0308
Epoch 9/100
15/15 [==============================] - 0s 11ms/step - loss: 0.0349 - val_loss: 0.0302
Epoch 10/100
15/15 [==============================] - 0s 10ms/step - loss: 0.0301 - val_loss: 0.0247

In [20]:
# Evaluate the model on the test data
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss with all features: {test_loss}')

# Predict on test data
y_pred = model.predict(X_test)

# Inverse scale the predictions and target values
y_pred = target_scaler.inverse_transform(y_pred)
y_test = target_scaler.inverse_transform(y_test)

# Check for NaN values in predictions
print(f"NaN in y_pred: {np.isnan(y_pred).sum()}")

# Remove NaN values
mask = ~np.isnan(y_pred).flatten()
y_test_filtered = y_test[mask]
y_pred_filtered = y_pred[mask]

# Debugging: Check shapes of y_test_filtered and y_pred_filtered
print("y_test_filtered shape:", y_test_filtered.shape)
print("y_pred_filtered shape:", y_pred_filtered.shape)

# Calculate mean absolute error (MAE)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test_filtered, y_pred_filtered)
print(f'Mean Absolute Error with all features: {mae}')


5/5 [==============================] - 0s 3ms/step - loss: 0.0200
Test Loss with all features: 0.020013101398944855
5/5 [==============================] - 0s 3ms/step
NaN in y_pred: 0
y_test_filtered shape: (146, 1)
y_pred_filtered shape: (146, 1)
Mean Absolute Error with all features: 0.6287282043726254


In [21]:
# Select features without past device temperature
features_no_device_temp = ['Temperature', 'Humidity', 'hour', 'day', 'month']
scaled_features_no_device_temp = feature_scaler.fit_transform(merged_df[features_no_device_temp])

# Prepare sequences for LSTM without device temperature
X_no_device_temp, y_no_device_temp = create_sequences(scaled_features_no_device_temp, scaled_target, seq_length)

# Split the data
X_train_no_device_temp, X_test_no_device_temp, y_train_no_device_temp, y_test_no_device_temp = train_test_split(
    X_no_device_temp, y_no_device_temp, test_size=0.2, random_state=42)

# Reshape input
X_train_no_device_temp = X_train_no_device_temp.reshape((X_train_no_device_temp.shape[0], seq_length, len(features_no_device_temp)))
X_test_no_device_temp = X_test_no_device_temp.reshape((X_test_no_device_temp.shape[0], seq_length, len(features_no_device_temp)))

# Define and compile the model
model_no_device_temp = Sequential([
    LSTM(128, activation='relu', return_sequences=True, input_shape=(seq_length, len(features_no_device_temp))),
    Dropout(0.3),
    LSTM(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)
])

optimizer = Adam(learning_rate=0.001)
model_no_device_temp.compile(optimizer=optimizer, loss='mse')

# Train the model
history_no_device_temp = model_no_device_temp.fit(X_train_no_device_temp, y_train_no_device_temp, epochs=30, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss_no_device_temp = model_no_device_temp.evaluate(X_test_no_device_temp, y_test_no_device_temp)
print(f'Test Loss without past device temperature: {test_loss_no_device_temp}')

# Predict on test data
y_pred_no_device_temp = model_no_device_temp.predict(X_test_no_device_temp)

# Inverse scale the predictions and target values
y_pred_no_device_temp = target_scaler.inverse_transform(y_pred_no_device_temp)
y_test_no_device_temp = target_scaler.inverse_transform(y_test_no_device_temp)

# Calculate mean absolute error (MAE)
mae_no_device_temp = mean_absolute_error(y_test_no_device_temp, y_pred_no_device_temp)
print(f'Mean Absolute Error without past device temperature: {mae_no_device_temp}')


Epoch 1/30
15/15 [==============================] - 2s 24ms/step - loss: 0.1634 - val_loss: 0.0919
Epoch 2/30
15/15 [==============================] - 0s 12ms/step - loss: 0.0932 - val_loss: 0.0915
Epoch 3/30
15/15 [==============================] - 0s 12ms/step - loss: 0.0842 - val_loss: 0.0996
Epoch 4/30
15/15 [==============================] - 0s 14ms/step - loss: 0.0775 - val_loss: 0.0831
Epoch 5/30
15/15 [==============================] - 0s 12ms/step - loss: 0.0676 - val_loss: 0.0820
Epoch 6/30
15/15 [==============================] - 0s 12ms/step - loss: 0.0647 - val_loss: 0.0860
Epoch 7/30
15/15 [==============================] - 0s 13ms/step - loss: 0.0625 - val_loss: 0.0809
Epoch 8/30
15/15 [==============================] - 0s 12ms/step - loss: 0.0576 - val_loss: 0.0813
Epoch 9/30
15/15 [==============================] - 0s 12ms/step - loss: 0.0583 - val_loss: 0.0855
Epoch 10/30
15/15 [==============================] - 0s 12ms/step - loss: 0.0587 - val_loss: 0.0816
Epoch 11/

In [22]:
# Select features without climate temperature
features_no_climate_temp = ['hour', 'day', 'month', 'temperature']
scaled_features_no_climate_temp = feature_scaler.fit_transform(merged_df[features_no_climate_temp])

# Prepare sequences for LSTM without climate temperature
X_no_climate_temp, y_no_climate_temp = create_sequences(scaled_features_no_climate_temp, scaled_target, seq_length)

# Split the data
X_train_no_climate_temp, X_test_no_climate_temp, y_train_no_climate_temp, y_test_no_climate_temp = train_test_split(
    X_no_climate_temp, y_no_climate_temp, test_size=0.2, random_state=42)

# Reshape input
X_train_no_climate_temp = X_train_no_climate_temp.reshape((X_train_no_climate_temp.shape[0], seq_length, len(features_no_climate_temp)))
X_test_no_climate_temp = X_test_no_climate_temp.reshape((X_test_no_climate_temp.shape[0], seq_length, len(features_no_climate_temp)))

# Define and compile the model
model_no_climate_temp = Sequential([
    LSTM(128, activation='relu', return_sequences=True, input_shape=(seq_length, len(features_no_climate_temp))),
    Dropout(0.3),
    LSTM(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)
])

optimizer = Adam(learning_rate=0.001)
model_no_climate_temp.compile(optimizer=optimizer, loss='mse')

# Train the model
history_no_climate_temp = model_no_climate_temp.fit(X_train_no_climate_temp, y_train_no_climate_temp, epochs=30, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss_no_climate_temp = model_no_climate_temp.evaluate(X_test_no_climate_temp, y_test_no_climate_temp)
print(f'Test Loss without climate temperature: {test_loss_no_climate_temp}')

# Predict on test data
y_pred_no_climate_temp = model_no_climate_temp.predict(X_test_no_climate_temp)

# Inverse scale the predictions and target values
y_pred_no_climate_temp = target_scaler.inverse_transform(y_pred_no_climate_temp)
y_test_no_climate_temp = target_scaler.inverse_transform(y_test_no_climate_temp)

# Calculate mean absolute error (MAE)
mae_no_climate_temp = mean_absolute_error(y_test_no_climate_temp, y_pred_no_climate_temp)
print(f'Mean Absolute Error without climate temperature: {mae_no_climate_temp}')


Epoch 1/30
15/15 [==============================] - 2s 20ms/step - loss: 0.1674 - val_loss: 0.0734
Epoch 2/30
15/15 [==============================] - 0s 9ms/step - loss: 0.0820 - val_loss: 0.0588
Epoch 3/30
15/15 [==============================] - 0s 9ms/step - loss: 0.0616 - val_loss: 0.0525
Epoch 4/30
15/15 [==============================] - 0s 10ms/step - loss: 0.0490 - val_loss: 0.0402
Epoch 5/30
15/15 [==============================] - 0s 10ms/step - loss: 0.0465 - val_loss: 0.0354
Epoch 6/30
15/15 [==============================] - 0s 9ms/step - loss: 0.0423 - val_loss: 0.0365
Epoch 7/30
15/15 [==============================] - 0s 9ms/step - loss: 0.0386 - val_loss: 0.0442
Epoch 8/30
15/15 [==============================] - 0s 9ms/step - loss: 0.0327 - val_loss: 0.0358
Epoch 9/30
15/15 [==============================] - 0s 9ms/step - loss: 0.0347 - val_loss: 0.0367
Epoch 10/30
15/15 [==============================] - 0s 9ms/step - loss: 0.0350 - val_loss: 0.0296
Epoch 11/30
15/1